In [7]:
import numpy as np
import pandas as pd
from prophet import Prophet
import warnings
warnings.filterwarnings('ignore')
import itertools
import pyodbc
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error as smape
from dotenv.main import load_dotenv
import os

In [8]:
# load_dotenv()
# # Koneksi ke SQL Server
# server = os.environ['SERVER']
# database = os.environ['DATABASE']
# username = os.environ['USERNAME_1']
# password = os.environ['PASSWORD']
# cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
# cursor = cnxn.cursor()
# query = "SELECT * FROM dwh.DM_financial_position_unpivot_full;"

In [9]:
server = '10.3.4.139,1433'
database = 'dwh_prod'
username = 'developerptba'
password = 'LnfPYVFW1K2TAKf'

In [10]:
# load_dotenv()
# Koneksi ke SQL Server
# server = '10.3.4.99,1433'
# database = 'dwh_dev'
# username = 'cybertrend'
# password = 'cn70HJBQHRfsoOS'
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
query = "SELECT * FROM dwh.DM_financial_position_unpivot_full;"
df = pd.read_sql(query, cnxn)
df

,Date,bulan,tahun,Indicator,BreakDown,Unit,Country,Source,tipe,amount,last_updated,Updated_Date
0,2022-01-01,1,2022,None,(Placement)/Withdrawal of time deposit,None,None,None,Moderate,0.00,NaT,NaT
1,2022-01-01,1,2022,None,Acquisition of associates,None,None,None,Worst,0.00,NaT,NaT
2,2022-01-01,1,2022,None,AP Turnover,None,None,None,Optimize,0.00,NaT,NaT
3,2022-01-01,1,2022,None,AR Turnover,None,None,None,Actual,0.00,NaT,NaT
4,2022-01-01,1,2022,None,Average Domestic Price,None,None,None,Moderate,0.00,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
69019,2024-08-01,8,2024,External Data,Average Market Price IDR,IDR/Ton,None,None,RKAP,983539.31,2024-08-31 05:11:01,2024-10-08 05:00:07.107
69020,2024-08-01,8,2024,External Data,Average Market Price IDR,IDR/Ton,None,None,Optimize,0.00,2024-08-31 05:11:01,2024-10-08 05:00:07.107
69021,2024-08-01,8,2024,External Data,Average Market Price IDR,IDR/Ton,None,None,Moderate,0.00,2024-08-31 05:11:01,2024-10-08 05:00:07.107
69022,2024-08-01,8,2024,External Data,Average Market Price IDR,IDR/Ton,None,None,Worst,0.00,2024-08-31 05:11:01,2024-10-08 05:00:07.107


In [11]:
# df = pd.read_csv("D:/PT Bukit Asam/Document 2024/Script Python/DM_financial_position_unpivot_full.csv")

In [12]:
# filter pada Dashboard Overview (dibuat dalam list)
BD_Overview =['Total Production',
    'BAP',
    'Total Transport',
    'Total Qty of Coal Sales',
    'SR',
    'Coal Cash Cost',
    'Average Market Price IDR',
    'Cash and Deposit',
    'Total Revenue',
    'EBITDA After Minority',
    'Net Profit After Minority'
]

In [13]:
# filter data berdasrkan Breakdown Yang dipilih
df_overview_filter = df[df['BreakDown'].isin(BD_Overview)]

In [14]:
# sum total production dari BD = total Production + BAP
df_overview_filter['BreakDown2']= df_overview_filter['BreakDown'].apply(lambda x: 'Total Production' if x in ['Total Production','BAP'] else x)


In [15]:
# Filter data yag akan dilakukan model dengan jenis : Actual
# df_overview_actual : jenis : aktual
df_overview_actual = df_overview_filter[df_overview_filter['tipe'] == "Actual"]


In [16]:
# data diurutkan dari awal data
df_overview_date = df_overview_actual.sort_values(by='Date')

In [17]:
df_overview = df_overview_date.groupby(['Date','BreakDown2'])['amount'].sum().reset_index()

PRODUCTION

In [18]:
print("--"*60)
print("PRODUCTION")

------------------------------------------------------------------------------------------------------------------------
PRODUCTION


In [19]:
df_production = df_overview[df_overview['BreakDown2']=='Total Production'][['Date','amount']]
# Reset index
df_production = df_production.reset_index(drop=True)
df_production.columns = ['ds','y']
df_production['ds'] = pd.to_datetime(df_production['ds'])


In [20]:
# Kombinasi parameter untuk tuning
changepoint_prior_scales = [0.01, 0.02]
seasonality_modes = ['additive']
seasonality_prior_scales = [0.01, 0.05, 1.0]

# Membuat list kombinasi parameter
param_combinations = list(itertools.product(changepoint_prior_scales, seasonality_modes, seasonality_prior_scales))

# Fungsi untuk menghitung SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Menyimpan hasil tuning
best_smape = float('inf')
best_params = None

# Train dan test split
train_size = int(0.8 * len(df_production))
train_data = df_production[:train_size]
test_data = df_production[train_size:]

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_mode, seasonality_prior_scale in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        weekly_seasonality=False, 
        daily_seasonality=False,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_mode=seasonality_mode,
        seasonality_prior_scale=seasonality_prior_scale
    )
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    
    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='MS')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, seasonality_mode={seasonality_mode}, seasonality_prior_scale={seasonality_prior_scale}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_mode, seasonality_prior_scale)

print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, seasonality_mode={best_params[1]}, seasonality_prior_scale={best_params[2]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True, 
    weekly_seasonality=False, 
    daily_seasonality=False,
    changepoint_prior_scale=best_params[0],
    seasonality_mode=best_params[1],
    seasonality_prior_scale=best_params[2]
)
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(df_production)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=12, freq='MS')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_df = forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_df.columns = ['Date', 'Forecast', 'Lower Bound', 'Upper Bound']

# Membulatkan kolom Forecast, Lower Bound, dan Upper Bound ke dua angka di belakang koma
forecast_df['Forecast'] = forecast_df['Forecast'].round(2)
forecast_df['Lower Bound'] = forecast_df['Lower Bound'].round(2)
forecast_df['Upper Bound'] = forecast_df['Upper Bound'].round(2)

# forecast_df['Forecast'] = forecast_df['Forecast'].apply(lambda x: f"{x:,.2f}")
# forecast_df['Lower Bound'] = forecast_df['Lower Bound'].apply(lambda x: f"{x:,.2f}")
# forecast_df['Upper Bound'] = forecast_df['Upper Bound'].apply(lambda x: f"{x:,.2f}")

# Menampilkan DataFrame hasil prediksi
# forecast_df

# Menampilkan DataFrame hasil prediksi
# print(forecast_df)


10:41:56 - cmdstanpy - INFO - Chain [1] start processing
10:41:56 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.01, seasonality_mode=additive, seasonality_prior_scale=0.01: 19.26%


10:41:57 - cmdstanpy - INFO - Chain [1] start processing
10:41:57 - cmdstanpy - INFO - Chain [1] done processing
10:41:57 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.01, seasonality_mode=additive, seasonality_prior_scale=0.05: 7.19%


10:42:00 - cmdstanpy - INFO - Chain [1] done processing
10:42:00 - cmdstanpy - INFO - Chain [1] start processing


...

SMAPE dengan changepoint_prior_scale=0.02, seasonality_mode=additive, seasonality_prior_scale=0.05: 7.19%


10:42:04 - cmdstanpy - INFO - Chain [1] done processing
10:42:04 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.02, seasonality_mode=additive, seasonality_prior_scale=1.0: 106.20%

Parameter terbaik: changepoint_prior_scale=0.02, seasonality_mode=additive, seasonality_prior_scale=0.05
SMAPE terbaik: 7.19%


10:42:04 - cmdstanpy - INFO - Chain [1] done processing


           ds          yhat    yhat_lower    yhat_upper
0  2022-01-01  1.927656e+06  1.654599e+06  2.196873e+06
1  2022-02-01  1.971304e+06  1.707445e+06  2.223274e+06
2  2022-03-01  2.374705e+06  2.128457e+06  2.638895e+06
3  2022-04-01  2.920710e+06  2.684828e+06  3.189278e+06
4  2022-05-01  3.278962e+06  3.014652e+06  3.537317e+06
5  2022-06-01  3.637217e+06  3.390737e+06  3.890411e+06
6  2022-07-01  3.902799e+06  3.641703e+06  4.152375e+06
7  2022-08-01  4.049390e+06  3.775115e+06  4.320147e+06
8  2022-09-01  3.917030e+06  3.648888e+06  4.156832e+06
9  2022-10-01  3.859806e+06  3.600310e+06  4.134710e+06
10 2022-11-01  2.749007e+06  2.487838e+06  2.992093e+06
11 2022-12-01  2.567903e+06  2.302627e+06  2.818753e+06
12 2023-01-01  1.996113e+06  1.736081e+06  2.235271e+06
13 2023-02-01  1.972033e+06  1.721193e+06  2.236021e+06
14 2023-03-01  2.897773e+06  2.640163e+06  3.161359e+06
15 2023-04-01  3.363472e+06  3.096432e+06  3.640495e+06
16 2023-05-01  3.807660e+06  3.540292e+06  4.073

In [21]:
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'ds': forecast_df['Date'],
    'y': forecast_df['Forecast'],
    'jenis': 'forecast',
    'Breakdown' : 'Total Production'
})

# Menambahkan data untuk 'Lower Bound'
lower_bound_rows = pd.DataFrame({
    'ds': forecast_df['Date'],
    'y': forecast_df['Lower Bound'],
    'jenis': 'lower bound',
    'Breakdown' : 'Total Production'
})

# Menambahkan data untuk 'Upper Bound'
upper_bound_rows = pd.DataFrame({
    'ds': forecast_df['Date'],
    'y': forecast_df['Upper Bound'],
    'jenis': 'upper bound',
    'Breakdown' : 'Total Production'
})
df_production_actual = pd.DataFrame({
    'ds': df_production['ds'],
    'y': df_production['y'],
    'jenis': 'actual',
    'Breakdown' : 'Total Production'
})
# Menggabungkan semua DataFrame
result_df_production = pd.concat([df_production_actual,forecast_rows, lower_bound_rows, upper_bound_rows], ignore_index=True)

# Menampilkan hasil akhir
# print(result_df_production)

---

In [22]:
print("--"*60)
print("SR")

------------------------------------------------------------------------------------------------------------------------
SR


SR

In [23]:
df_SR = df_overview[df_overview['BreakDown2']=='SR'][['Date','amount']]
# Reset index
df_SR = df_SR.reset_index(drop=True)
df_SR.columns = ['ds','y']
df_SR['ds'] = pd.to_datetime(df_SR['ds'])

In [24]:
# Kombinasi parameter untuk tuning
changepoint_prior_scales = [0.05, 0.1]
seasonality_modes = ['additive']
seasonality_prior_scales = [0.01, 0.05, 1.0]

# Membuat list kombinasi parameter
param_combinations = list(itertools.product(changepoint_prior_scales, seasonality_modes, seasonality_prior_scales))

# Fungsi untuk menghitung SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Menyimpan hasil tuning
best_smape = float('inf')
best_params = None

# Train dan test split
train_size = int(0.8 * len(df_SR))
train_data = df_SR[:train_size]
test_data = df_SR[train_size:]

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_mode, seasonality_prior_scale in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        weekly_seasonality=False, 
        daily_seasonality=False,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_mode=seasonality_mode,
        seasonality_prior_scale=seasonality_prior_scale
    )
    model.add_seasonality(name='monthly', period=30.5, fourier_order=3)
    
    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='MS')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, seasonality_mode={seasonality_mode}, seasonality_prior_scale={seasonality_prior_scale}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_mode, seasonality_prior_scale)

print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, seasonality_mode={best_params[1]}, seasonality_prior_scale={best_params[2]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True, 
    weekly_seasonality=False, 
    daily_seasonality=False,
    changepoint_prior_scale=best_params[0],
    seasonality_mode=best_params[1],
    seasonality_prior_scale=best_params[2]
)
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(df_SR)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=12, freq='MS')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_df_SR = forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_df_SR.columns = ['Date', 'Forecast', 'Lower Bound', 'Upper Bound']

forecast_df_SR['Forecast'] = forecast_df_SR['Forecast'].round(2)
forecast_df_SR['Lower Bound'] = forecast_df_SR['Lower Bound'].round(2)
forecast_df_SR['Upper Bound'] = forecast_df_SR['Upper Bound'].round(2)

# Menampilkan DataFrame hasil prediksi
# print(forecast_df_SR)
# forecast_df_SR.tail(16)


10:42:05 - cmdstanpy - INFO - Chain [1] start processing
10:42:05 - cmdstanpy - INFO - Chain [1] done processing
10:42:05 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.05, seasonality_mode=additive, seasonality_prior_scale=0.01: 13.62%


10:42:05 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.05, seasonality_mode=additive, seasonality_prior_scale=0.05: 9.46%


10:42:06 - cmdstanpy - INFO - Chain [1] start processing
10:42:06 - cmdstanpy - INFO - Chain [1] done processing
10:42:06 - cmdstanpy - INFO - Chain [1] start processing


...

SMAPE dengan changepoint_prior_scale=0.1, seasonality_mode=additive, seasonality_prior_scale=0.05: 9.45%


10:42:08 - cmdstanpy - INFO - Chain [1] done processing
10:42:08 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.1, seasonality_mode=additive, seasonality_prior_scale=1.0: 38.76%

Parameter terbaik: changepoint_prior_scale=0.1, seasonality_mode=additive, seasonality_prior_scale=0.05
SMAPE terbaik: 9.45%


10:42:08 - cmdstanpy - INFO - Chain [1] done processing


           ds      yhat  yhat_lower  yhat_upper
0  2022-01-01  6.488482    5.803370    7.144544
1  2022-02-01  6.550897    5.924817    7.219324
2  2022-03-01  5.707338    5.054493    6.392973
3  2022-04-01  5.334345    4.648375    6.028585
4  2022-05-01  5.292303    4.500592    6.023279
5  2022-06-01  5.107715    4.470787    5.876740
6  2022-07-01  5.316719    4.568189    6.040201
7  2022-08-01  5.612665    4.914174    6.308478
8  2022-09-01  5.517243    4.874386    6.243179
9  2022-10-01  5.670939    5.028656    6.448203
10 2022-11-01  7.603851    6.932142    8.282967
11 2022-12-01  7.398634    6.726624    8.098372
12 2023-01-01  6.878953    6.196963    7.588796
13 2023-02-01  6.996143    6.330536    7.723407
14 2023-03-01  5.917448    5.224538    6.617508
15 2023-04-01  5.565407    4.833026    6.316793
16 2023-05-01  5.493918    4.782264    6.187913
17 2023-06-01  5.329507    4.671901    6.028903
18 2023-07-01  5.507733    4.833665    6.155530
19 2023-08-01  5.819898    5.082795    6

In [25]:
forecast_df_SR.loc[:, 'Date'] = pd.to_datetime(forecast_df_SR['Date'])
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'ds': forecast_df_SR['Date'],
    'y': forecast_df_SR['Forecast'],
    'jenis': 'forecast',
    'Breakdown' : 'SR'
})

# Menambahkan data untuk 'Lower Bound'
lower_bound_rows = pd.DataFrame({
    'ds': forecast_df_SR['Date'],
    'y': forecast_df_SR['Lower Bound'],
    'jenis': 'lower bound',
    'Breakdown' : 'SR'
})

# Menambahkan data untuk 'Upper Bound'
upper_bound_rows = pd.DataFrame({
    'ds': forecast_df_SR['Date'],
    'y': forecast_df_SR['Upper Bound'],
    'jenis': 'upper bound',
    'Breakdown' : 'SR'
})
df_SR_actual = pd.DataFrame({
    'ds': df_SR['ds'],
    'y': df_SR['y'],
    'jenis': 'actual',
    'Breakdown' : 'SR'
})
# Menggabungkan semua DataFrame
result_df_SR = pd.concat([df_SR_actual,forecast_rows, lower_bound_rows, upper_bound_rows], ignore_index=True)

# Menampilkan hasil akhir
# result_df_SR

-----

In [26]:
print("--"*60)
print("TRANSPORT")

------------------------------------------------------------------------------------------------------------------------
TRANSPORT


In [27]:
df_transport = df_overview[df_overview['BreakDown2']=='Total Transport'][['Date','amount']]
# Reset index
df_transport = df_transport.reset_index(drop=True)
df_transport.columns = ['ds','y']
df_transport['ds'] = pd.to_datetime(df_transport['ds'])

In [28]:
# Kombinasi parameter untuk tuning SARIMA
p = d = q = range(0, 2)  # AR, I, MA order
P = D = Q = range(0, 2)  # Seasonal ARIMA order
s = [12]  # Musiman dalam bulan (12 bulan)

# Membuat list kombinasi parameter p, d, q, P, D, Q, s
param_combinations = list(itertools.product(p, d, q, P, D, Q, s))

# Menyimpan hasil tuning
best_smape = float('inf')
best_params = None

# Train dan test split
train_size = int(0.8 * len(df_transport))
train_data = df_transport[:train_size]
test_data = df_transport[train_size:]

# Fungsi SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Looping untuk setiap kombinasi parameter
for (p, d, q, P, D, Q, s) in param_combinations:
    try:
        # Inisialisasi model SARIMA dengan parameter tuning
        model = SARIMAX(train_data['y'],
                        order=(p, d, q),
                        seasonal_order=(P, D, Q, s),
                        enforce_stationarity=False,
                        enforce_invertibility=False)

        # Melatih model dengan data train
        model_fit = model.fit(disp=False)

        # Melakukan prediksi
        forecast = model_fit.forecast(steps=len(test_data))

        # Menggabungkan data asli dengan prediksi
        test_data.loc[:, 'yhat'] = forecast

        # Menghitung SMAPE untuk data test
        smape_value = smape(test_data['y'], test_data['yhat'])
        print(f"SMAPE dengan order=({p},{d},{q}), seasonal_order=({P},{D},{Q},{s}): {smape_value:.2f}%")

        # Menyimpan kombinasi parameter terbaik
        if smape_value < best_smape:
            best_smape = smape_value
            best_params = (p, d, q, P, D, Q, s)

    except Exception as e:
        print(f"Error untuk parameter {p, d, q, P, D, Q, s}: {e}")

print(f"\nParameter terbaik: order=({best_params[0]},{best_params[1]},{best_params[2]}), seasonal_order=({best_params[3]},{best_params[4]},{best_params[5]},{best_params[6]})")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Membuat model SARIMAX menggunakan parameter terbaik
final_model = SARIMAX(df_transport['y'],
                     order=(best_params[0], best_params[1], best_params[2]),
                     seasonal_order=(best_params[3], best_params[4], best_params[5], best_params[6]),
                     enforce_stationarity=False,
                     enforce_invertibility=False)

# Melatih model dengan seluruh data
final_model_fit = final_model.fit(disp=False)

# Prediksi untuk seluruh data historis (dari awal data hingga akhir)
forecast_historic = final_model_fit.get_prediction(steps=len(df_transport))
forecast_historic_mean = forecast_historic.predicted_mean
forecast_historic_ci = forecast_historic.conf_int()

# Prediksi untuk 12 bulan ke depan
forecast_future = final_model_fit.get_forecast(steps=12)
forecast_future_mean = forecast_future.predicted_mean
forecast_future_ci = forecast_future.conf_int()

# Membuat tanggal untuk prediksi masa depan
future_dates = pd.date_range(start=df_transport['ds'].max() + pd.DateOffset(months=1), periods=12, freq='MS')

# Konversi 'future_dates' ke Series agar bisa digabungkan dengan tanggal historis
future_dates_series = pd.Series(future_dates)

# Menyimpan hasil prediksi ke dalam DataFrame untuk data historis dan data masa depan
forecast_df_transport = pd.DataFrame({
    'Date': pd.concat([df_transport['ds'].reset_index(drop=True), future_dates_series], ignore_index=True),
    'Forecast': pd.concat([forecast_historic_mean, forecast_future_mean], ignore_index=True),
    'Lower Bound': pd.concat([forecast_historic_ci.iloc[:, 0], forecast_future_ci.iloc[:, 0]], ignore_index=True),
    'Upper Bound': pd.concat([forecast_historic_ci.iloc[:, 1], forecast_future_ci.iloc[:, 1]], ignore_index=True)
})

forecast_df_transport['Forecast'] = forecast_df_transport['Forecast'].round(2)
forecast_df_transport['Lower Bound'] = forecast_df_transport['Lower Bound'].round(2)
forecast_df_transport['Upper Bound'] = forecast_df_transport['Upper Bound'].round(2)

# Menampilkan hasil prediksi untuk seluruh data (historis + masa depan)
# print(forecast_df_transport)

SMAPE dengan order=(0,0,0), seasonal_order=(0,0,0,12): 200.00%
SMAPE dengan order=(0,0,0), seasonal_order=(0,0,1,12): 188.99%
SMAPE dengan order=(0,0,0), seasonal_order=(0,1,0,12): 10.60%
SMAPE dengan order=(0,0,0), seasonal_order=(0,1,1,12): 10.60%
SMAPE dengan order=(0,0,0), seasonal_order=(1,0,0,12): 4.56%
SMAPE dengan order=(0,0,0), seasonal_order=(1,0,1,12): 67.08%
SMAPE dengan order=(0,0,0), seasonal_order=(1,1,0,12): 7.46%
SMAPE dengan order=(0,0,0), seasonal_order=(1,1,1,12): 10.60%
SMAPE dengan order=(0,0,1), seasonal_order=(0,0,0,12): 183.43%
SMAPE dengan order=(0,0,1), seasonal_order=(0,0,1,12): 188.01%
SMAPE dengan order=(0,0,1), seasonal_order=(0,1,0,12): 9.64%
SMAPE dengan order=(0,0,1), seasonal_order=(0,1,1,12): 9.77%
SMAPE dengan order=(0,0,1), seasonal_order=(1,0,0,12): 4.27%
SMAPE dengan order=(0,0,1), seasonal_order=(1,0,1,12): 195.57%
SMAPE dengan order=(0,0,1), seasonal_order=(1,1,0,12): 7.31%
SMAPE dengan order=(0,0,1), seasonal_order=(1,1,1,12): 9.77%
SMAPE deng

In [29]:
forecast_df_transport.loc[:, 'Date'] = pd.to_datetime(forecast_df_transport['Date'])
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'ds': forecast_df_transport['Date'],
    'y': forecast_df_transport['Forecast'],
    'jenis': 'forecast',
    'Breakdown' : 'Total Transport'
})

# Menambahkan data untuk 'Lower Bound'
lower_bound_rows = pd.DataFrame({
    'ds': forecast_df_transport['Date'],
    'y': forecast_df_transport['Lower Bound'],
    'jenis': 'lower bound',
    'Breakdown' : 'Total Transport'
})

# Menambahkan data untuk 'Upper Bound'
upper_bound_rows = pd.DataFrame({
    'ds': forecast_df_transport['Date'],
    'y': forecast_df_transport['Upper Bound'],
    'jenis': 'upper bound',
    'Breakdown' : 'Total Transport'
})
df_Transport_actual = pd.DataFrame({
    'ds': df_transport['ds'],
    'y': df_transport['y'],
    'jenis': 'actual',
    'Breakdown' : 'Total Transport'
})
# Menggabungkan semua DataFrame
result_df_Transport = pd.concat([df_Transport_actual,forecast_rows, lower_bound_rows, upper_bound_rows], ignore_index=True)

# Menampilkan hasil akhir
# result_df_Transport

---

In [30]:
print("--"*60)
print("SALES")

------------------------------------------------------------------------------------------------------------------------
SALES


In [31]:
df_Sales = df_overview[df_overview['BreakDown2']=='Total Qty of Coal Sales'][['Date','amount']]
# Reset index
df_Sales = df_Sales.reset_index(drop=True)
df_Sales.columns = ['ds','y']
df_Sales['ds'] = pd.to_datetime(df_Sales['ds'])


In [32]:
#  Kombinasi parameter untuk tuning SARIMA
p = d = q = range(0, 2)  # AR, I, MA order
P = D = Q = range(0, 2)  # Seasonal ARIMA order
s = [12]  # Musiman dalam bulan (12 bulan)

# Membuat list kombinasi parameter p, d, q, P, D, Q, s
param_combinations = list(itertools.product(p, d, q, P, D, Q, s))

# Menyimpan hasil tuning
best_smape = float('inf')
best_params = None

# Train dan test split
train_size = int(0.8 * len(df_Sales))
train_data = df_Sales[:train_size]
test_data = df_Sales[train_size:]

# Fungsi SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Looping untuk setiap kombinasi parameter
for (p, d, q, P, D, Q, s) in param_combinations:
    try:
        # Inisialisasi model SARIMA dengan parameter tuning
        model = SARIMAX(train_data['y'],
                        order=(p, d, q),
                        seasonal_order=(P, D, Q, s),
                        enforce_stationarity=False,
                        enforce_invertibility=False)

        # Melatih model dengan data train
        model_fit = model.fit(disp=False)

        # Melakukan prediksi
        forecast = model_fit.forecast(steps=len(test_data))

        # Menggabungkan data asli dengan prediksi
        test_data.loc[:, 'yhat'] = forecast

        # Menghitung SMAPE untuk data test
        smape_value = smape(test_data['y'], test_data['yhat'])
        print(f"SMAPE dengan order=({p},{d},{q}), seasonal_order=({P},{D},{Q},{s}): {smape_value:.2f}%")

        # Menyimpan kombinasi parameter terbaik
        if smape_value < best_smape:
            best_smape = smape_value
            best_params = (p, d, q, P, D, Q, s)

    except Exception as e:
        print(f"Error untuk parameter {p, d, q, P, D, Q, s}: {e}")

print(f"\nParameter terbaik: order=({best_params[0]},{best_params[1]},{best_params[2]}), seasonal_order=({best_params[3]},{best_params[4]},{best_params[5]},{best_params[6]})")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Membuat model SARIMAX menggunakan parameter terbaik
final_model = SARIMAX(df_Sales['y'],
                     order=(best_params[0], best_params[1], best_params[2]),
                     seasonal_order=(best_params[3], best_params[4], best_params[5], best_params[6]),
                     enforce_stationarity=False,
                     enforce_invertibility=False)

# Melatih model dengan seluruh data
final_model_fit = final_model.fit(disp=False)

# Prediksi untuk seluruh data historis (dari awal data hingga akhir)
forecast_historic = final_model_fit.get_prediction(steps=len(df_Sales))
forecast_historic_mean = forecast_historic.predicted_mean
forecast_historic_ci = forecast_historic.conf_int()

# Prediksi untuk 12 bulan ke depan
forecast_future = final_model_fit.get_forecast(steps=12)
forecast_future_mean = forecast_future.predicted_mean
forecast_future_ci = forecast_future.conf_int()

# Membuat tanggal untuk prediksi masa depan
future_dates = pd.date_range(start=df_Sales['ds'].max() + pd.DateOffset(months=1), periods=12, freq='MS')

# Konversi 'future_dates' ke Series agar bisa digabungkan dengan tanggal historis
future_dates_series = pd.Series(future_dates)

# Menyimpan hasil prediksi ke dalam DataFrame untuk data historis dan data masa depan
forecast_df_Sales = pd.DataFrame({
    'Date': pd.concat([df_Sales['ds'].reset_index(drop=True), future_dates_series], ignore_index=True),
    'Forecast': pd.concat([forecast_historic_mean, forecast_future_mean], ignore_index=True),
    'Lower Bound': pd.concat([forecast_historic_ci.iloc[:, 0], forecast_future_ci.iloc[:, 0]], ignore_index=True),
    'Upper Bound': pd.concat([forecast_historic_ci.iloc[:, 1], forecast_future_ci.iloc[:, 1]], ignore_index=True)
})

forecast_df_Sales['Forecast'] = forecast_df_Sales['Forecast'].round(2)
forecast_df_Sales['Lower Bound'] = forecast_df_Sales['Lower Bound'].round(2)
forecast_df_Sales['Upper Bound'] = forecast_df_Sales['Upper Bound'].round(2)

# Menampilkan hasil prediksi untuk seluruh data (historis + masa depan)
# forecast_df_Sales

SMAPE dengan order=(0,0,0), seasonal_order=(0,0,0,12): 200.00%
SMAPE dengan order=(0,0,0), seasonal_order=(0,0,1,12): 181.90%
SMAPE dengan order=(0,0,0), seasonal_order=(0,1,0,12): 17.00%
SMAPE dengan order=(0,0,0), seasonal_order=(0,1,1,12): 17.00%
SMAPE dengan order=(0,0,0), seasonal_order=(1,0,0,12): 9.28%
SMAPE dengan order=(0,0,0), seasonal_order=(1,0,1,12): 69.28%
SMAPE dengan order=(0,0,0), seasonal_order=(1,1,0,12): 14.57%
SMAPE dengan order=(0,0,0), seasonal_order=(1,1,1,12): 17.00%
SMAPE dengan order=(0,0,1), seasonal_order=(0,0,0,12): 180.82%
SMAPE dengan order=(0,0,1), seasonal_order=(0,0,1,12): 181.88%
SMAPE dengan order=(0,0,1), seasonal_order=(0,1,0,12): 16.86%
SMAPE dengan order=(0,0,1), seasonal_order=(0,1,1,12): 17.03%
SMAPE dengan order=(0,0,1), seasonal_order=(1,0,0,12): 9.84%
SMAPE dengan order=(0,0,1), seasonal_order=(1,0,1,12): 14.30%
SMAPE dengan order=(0,0,1), seasonal_order=(1,1,0,12): 14.73%
SMAPE dengan order=(0,0,1), seasonal_order=(1,1,1,12): 17.03%
SMAPE 

In [33]:
forecast_df_Sales

,Date,Forecast,Lower Bound,Upper Bound
0,2022-01-01,0.00,-1959.96,1959.96
1,2022-02-01,2120703.83,1508884.62,2732523.04
2,2022-03-01,2151628.62,1539806.04,2763451.20
3,2022-04-01,2166193.88,1554367.67,2778020.09
4,2022-05-01,2183362.07,1571531.95,2795192.18
5,2022-06-01,2202868.91,1591034.60,2814703.21
6,2022-07-01,2231830.72,1619991.91,2843669.53
7,2022-08-01,2254256.58,1642412.92,2866100.23
8,2022-09-01,2269338.02,1657489.15,2881186.88
9,2022-10-01,2277003.35,1665148.88,2888857.81


In [34]:
forecast_df_Sales.loc[:, 'Date'] = pd.to_datetime(forecast_df_Sales['Date'])
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'ds': forecast_df_Sales['Date'],
    'y': forecast_df_Sales['Forecast'],
    'jenis': 'forecast',
    'Breakdown' : 'Total Qty of Coal Sales'
})

# Menambahkan data untuk 'Lower Bound'
lower_bound_rows = pd.DataFrame({
    'ds': forecast_df_Sales['Date'],
    'y': forecast_df_Sales['Lower Bound'],
    'jenis': 'lower bound',
    'Breakdown' : 'Total Qty of Coal Sales'
})

# Menambahkan data untuk 'Upper Bound'
upper_bound_rows = pd.DataFrame({
    'ds': forecast_df_Sales['Date'],
    'y': forecast_df_Sales['Upper Bound'],
    'jenis': 'upper bound',
    'Breakdown' : 'Total Qty of Coal Sales'
})
df_Sales_actual = pd.DataFrame({
    'ds': df_Sales['ds'],
    'y': df_Sales['y'],
    'jenis': 'actual',
    'Breakdown' : 'Total Qty of Coal Sales'
})
# Menggabungkan semua DataFrame
result_df_Sales = pd.concat([df_Sales_actual,forecast_rows, lower_bound_rows, upper_bound_rows], ignore_index=True)

# Menampilkan hasil akhir
# result_df_Sales

---

In [35]:
print("--"*60)
print("CASH COST")

------------------------------------------------------------------------------------------------------------------------
CASH COST


In [36]:
df_coal_cast_cost = df_overview[df_overview['BreakDown2']=='Coal Cash Cost'][['Date','amount']]
# Reset index
df_coal_cast_cost = df_coal_cast_cost.reset_index(drop=True)
df_coal_cast_cost.columns = ['ds','y']
df_coal_cast_cost['ds'] = pd.to_datetime(df_coal_cast_cost['ds'])

In [37]:
# Kombinasi parameter untuk tuning
changepoint_prior_scales = [0.05, 0.1, 0.2]
seasonality_modes = ['additive']
seasonality_prior_scales = [0.1, 0.2]

# Membuat list kombinasi parameter
param_combinations = list(itertools.product(changepoint_prior_scales, seasonality_modes, seasonality_prior_scales))

# Fungsi untuk menghitung SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Menyimpan hasil tuning
best_smape = float('inf')
best_params = None

# Train dan test split
train_size = int(0.8 * len(df_coal_cast_cost))
train_data = df_coal_cast_cost[:train_size]
test_data = df_coal_cast_cost[train_size:]

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_mode, seasonality_prior_scale in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        # weekly_seasonality=False, 
        # daily_seasonality=False,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_mode=seasonality_mode,
        seasonality_prior_scale=seasonality_prior_scale
    )
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    
    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='MS')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, seasonality_mode={seasonality_mode}, seasonality_prior_scale={seasonality_prior_scale}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_mode, seasonality_prior_scale)

print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, seasonality_mode={best_params[1]}, seasonality_prior_scale={best_params[2]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True, 
    # weekly_seasonality=False, 
    # daily_seasonality=False,
    changepoint_prior_scale=best_params[0],
    seasonality_mode=best_params[1],
    seasonality_prior_scale=best_params[2]
)
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(df_coal_cast_cost)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=12, freq='MS')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_df_coal_cast_cost = forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_df_coal_cast_cost.columns = ['Date', 'Forecast', 'Lower Bound', 'Upper Bound']

forecast_df_coal_cast_cost['Forecast'] = forecast_df_coal_cast_cost['Forecast'].round(2)
forecast_df_coal_cast_cost['Lower Bound'] = forecast_df_coal_cast_cost['Lower Bound'].round(2)
forecast_df_coal_cast_cost['Upper Bound'] = forecast_df_coal_cast_cost['Upper Bound'].round(2)

# Menampilkan DataFrame hasil prediksi
# forecast_df_coal_cast_cost.tail(16)


10:42:12 - cmdstanpy - INFO - Chain [1] start processing
10:42:12 - cmdstanpy - INFO - Chain [1] done processing
10:42:12 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.05, seasonality_mode=additive, seasonality_prior_scale=0.1: 25.50%


10:42:12 - cmdstanpy - INFO - Chain [1] done processing
10:42:13 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.05, seasonality_mode=additive, seasonality_prior_scale=0.2: 20.45%


10:42:13 - cmdstanpy - INFO - Chain [1] done processing
10:42:13 - cmdstanpy - INFO - Chain [1] start processing


...

SMAPE dengan changepoint_prior_scale=0.2, seasonality_mode=additive, seasonality_prior_scale=0.1: 38.12%


10:42:15 - cmdstanpy - INFO - Chain [1] done processing
10:42:15 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.2, seasonality_mode=additive, seasonality_prior_scale=0.2: 43.64%

Parameter terbaik: changepoint_prior_scale=0.05, seasonality_mode=additive, seasonality_prior_scale=0.2
SMAPE terbaik: 20.45%


10:42:15 - cmdstanpy - INFO - Chain [1] done processing


           ds          yhat    yhat_lower    yhat_upper
0  2022-01-01  7.960926e+05  7.090483e+05  8.860699e+05
1  2022-02-01  6.751737e+05  5.912012e+05  7.621983e+05
2  2022-03-01  8.613742e+05  7.709040e+05  9.500627e+05
3  2022-04-01  9.126425e+05  8.282633e+05  1.002942e+06
4  2022-05-01  7.948214e+05  7.059860e+05  8.823043e+05
5  2022-06-01  7.740566e+05  6.884020e+05  8.665359e+05
6  2022-07-01  7.801596e+05  6.950889e+05  8.740957e+05
7  2022-08-01  8.543820e+05  7.661570e+05  9.413997e+05
8  2022-09-01  8.748044e+05  7.837609e+05  9.608741e+05
9  2022-10-01  9.445945e+05  8.553273e+05  1.037252e+06
10 2022-11-01  9.481304e+05  8.611513e+05  1.038764e+06
11 2022-12-01  9.377848e+05  8.503477e+05  1.025550e+06
12 2023-01-01  9.099961e+05  8.212559e+05  9.943847e+05
13 2023-02-01  9.121438e+05  8.292095e+05  1.003677e+06
14 2023-03-01  8.461432e+05  7.575378e+05  9.357014e+05
15 2023-04-01  8.355705e+05  7.469630e+05  9.200034e+05
16 2023-05-01  7.780674e+05  6.963426e+05  8.671

In [38]:
forecast_df_coal_cast_cost.loc[:, 'Date'] = pd.to_datetime(forecast_df_coal_cast_cost['Date'])
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'ds': forecast_df_coal_cast_cost['Date'],
    'y': forecast_df_coal_cast_cost['Forecast'],
    'jenis': 'forecast',
    'Breakdown' : 'Coal Cash Cost'
})

# Menambahkan data untuk 'Lower Bound'
lower_bound_rows = pd.DataFrame({
    'ds': forecast_df_coal_cast_cost['Date'],
    'y': forecast_df_coal_cast_cost['Lower Bound'],
    'jenis': 'lower bound',
    'Breakdown' : 'Coal Cash Cost'
})

# Menambahkan data untuk 'Upper Bound'
upper_bound_rows = pd.DataFrame({
    'ds': forecast_df_coal_cast_cost['Date'],
    'y': forecast_df_coal_cast_cost['Upper Bound'],
    'jenis': 'upper bound',
    'Breakdown' : 'Coal Cash Cost'
})
df_coal_cast_cost_actual = pd.DataFrame({
    'ds': df_coal_cast_cost['ds'],
    'y': df_coal_cast_cost['y'],
    'jenis': 'actual',
    'Breakdown' : 'Coal Cash Cost'
})
# Menggabungkan semua DataFrame
result_df_coal_cast_cost = pd.concat([df_coal_cast_cost_actual,forecast_rows, lower_bound_rows, upper_bound_rows], ignore_index=True)

# Menampilkan hasil akhir
# result_df_coal_cast_cost

---

In [39]:
print("--"*60)
print("AVG. SELLING PRICE")

------------------------------------------------------------------------------------------------------------------------
AVG. SELLING PRICE


In [40]:
df_AvgSP = df_overview[df_overview['BreakDown2']=='Average Market Price IDR'][['Date','amount']]
# Reset index
df_AvgSP = df_AvgSP.reset_index(drop=True)
df_AvgSP.columns = ['ds','y']
df_AvgSP['ds'] = pd.to_datetime(df_AvgSP['ds'])


In [41]:
# Kombinasi parameter untuk tuning
changepoint_prior_scales = [0.05, 0.1, 0.2]
seasonality_modes = ['additive','multiplicative']
seasonality_prior_scales = [0.1, 0.2]

# Membuat list kombinasi parameter
param_combinations = list(itertools.product(changepoint_prior_scales, seasonality_modes, seasonality_prior_scales))

# Fungsi untuk menghitung SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Menyimpan hasil tuning
best_smape = float('inf')
best_params = None

# Train dan test split
train_size = int(0.8 * len(df_AvgSP))
train_data = df_AvgSP[:train_size]
test_data = df_AvgSP[train_size:]

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_mode, seasonality_prior_scale in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        # weekly_seasonality=False, 
        # daily_seasonality=False,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_mode=seasonality_mode,
        seasonality_prior_scale=seasonality_prior_scale
    )
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    
    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='MS')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, seasonality_mode={seasonality_mode}, seasonality_prior_scale={seasonality_prior_scale}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_mode, seasonality_prior_scale)

print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, seasonality_mode={best_params[1]}, seasonality_prior_scale={best_params[2]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True, 
    # weekly_seasonality=False, 
    # daily_seasonality=False,
    changepoint_prior_scale=best_params[0],
    seasonality_mode=best_params[1],
    seasonality_prior_scale=best_params[2]
)
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(df_AvgSP)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=12, freq='MS')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_df_AvgSP = forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_df_AvgSP.columns = ['Date', 'Forecast', 'Lower Bound', 'Upper Bound']

forecast_df_AvgSP['Forecast'] = forecast_df_AvgSP['Forecast'].round(2)
forecast_df_AvgSP['Lower Bound'] = forecast_df_AvgSP['Lower Bound'].round(2)
forecast_df_AvgSP['Upper Bound'] = forecast_df_AvgSP['Upper Bound'].round(2)

# Menampilkan DataFrame hasil prediksi
# forecast_df_AvgSP.tail(16)


10:42:15 - cmdstanpy - INFO - Chain [1] start processing
10:42:16 - cmdstanpy - INFO - Chain [1] done processing
10:42:16 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.05, seasonality_mode=additive, seasonality_prior_scale=0.1: 16.48%


10:42:16 - cmdstanpy - INFO - Chain [1] done processing
10:42:16 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.05, seasonality_mode=additive, seasonality_prior_scale=0.2: 16.32%


10:42:17 - cmdstanpy - INFO - Chain [1] done processing
10:42:17 - cmdstanpy - INFO - Chain [1] start processing


...

SMAPE dengan changepoint_prior_scale=0.2, seasonality_mode=multiplicative, seasonality_prior_scale=0.1: 7.03%


10:42:21 - cmdstanpy - INFO - Chain [1] done processing
10:42:21 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.2, seasonality_mode=multiplicative, seasonality_prior_scale=0.2: 7.62%

Parameter terbaik: changepoint_prior_scale=0.2, seasonality_mode=multiplicative, seasonality_prior_scale=0.1
SMAPE terbaik: 7.03%


10:42:22 - cmdstanpy - INFO - Chain [1] done processing


           ds          yhat    yhat_lower    yhat_upper
0  2022-01-01  1.029162e+06  9.530023e+05  1.111951e+06
1  2022-02-01  9.954444e+05  9.129744e+05  1.072341e+06
2  2022-03-01  1.398842e+06  1.324210e+06  1.474505e+06
3  2022-04-01  1.360296e+06  1.278662e+06  1.441162e+06
4  2022-05-01  1.330623e+06  1.251784e+06  1.405600e+06
5  2022-06-01  1.309969e+06  1.230179e+06  1.389231e+06
6  2022-07-01  1.415541e+06  1.345713e+06  1.491231e+06
7  2022-08-01  1.419167e+06  1.343653e+06  1.498951e+06
8  2022-09-01  1.314703e+06  1.230572e+06  1.398442e+06
9  2022-10-01  1.537041e+06  1.454793e+06  1.609715e+06
10 2022-11-01  1.307939e+06  1.237224e+06  1.387712e+06
11 2022-12-01  1.353641e+06  1.275470e+06  1.431206e+06
12 2023-01-01  1.119012e+06  1.040886e+06  1.196387e+06
13 2023-02-01  1.053821e+06  9.741073e+05  1.130606e+06
14 2023-03-01  1.112513e+06  1.039002e+06  1.190097e+06
15 2023-04-01  1.092130e+06  1.016994e+06  1.166634e+06
16 2023-05-01  9.578317e+05  8.823790e+05  1.032

In [42]:
forecast_df_coal_cast_cost.loc[:, 'Date'] = pd.to_datetime(forecast_df_AvgSP['Date'])
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'ds': forecast_df_AvgSP['Date'],
    'y': forecast_df_AvgSP['Forecast'],
    'jenis': 'forecast',
    'Breakdown' : 'Average Market Price IDR'
})

# Menambahkan data untuk 'Lower Bound'
lower_bound_rows = pd.DataFrame({
    'ds': forecast_df_AvgSP['Date'],
    'y': forecast_df_AvgSP['Lower Bound'],
    'jenis': 'lower bound',
    'Breakdown' : 'Average Market Price IDR'
})

# Menambahkan data untuk 'Upper Bound'
upper_bound_rows = pd.DataFrame({
    'ds': forecast_df_AvgSP['Date'],
    'y': forecast_df_coal_cast_cost['Upper Bound'],
    'jenis': 'upper bound',
    'Breakdown' : 'Average Market Price IDR'
})
df_Avg_SP_actual = pd.DataFrame({
    'ds': df_AvgSP['ds'],
    'y': df_AvgSP['y'],
    'jenis': 'actual',
    'Breakdown' : 'Average Market Price IDR'
})
# Menggabungkan semua DataFrame
result_df_Avg_SP = pd.concat([df_Avg_SP_actual,forecast_rows, lower_bound_rows, upper_bound_rows], ignore_index=True)

# Menampilkan hasil akhir
# result_df_Avg_SP

---

In [43]:
print("--"*60)
print("REVENUE")

------------------------------------------------------------------------------------------------------------------------
REVENUE


In [44]:
df_Revenue = df_overview[df_overview['BreakDown2']=='Total Revenue'][['Date','amount']]
# Reset index
df_Revenue = df_Revenue.reset_index(drop=True)
df_Revenue.columns = ['ds','y']
df_Revenue['ds'] = pd.to_datetime(df_Revenue['ds'])

In [45]:
# Kombinasi parameter untuk tuning
changepoint_prior_scales = [0.05, 0.1, 0.2]
seasonality_modes = ['additive','multiplicative']
seasonality_prior_scales = [0.1, 0.2]

# Membuat list kombinasi parameter
param_combinations = list(itertools.product(changepoint_prior_scales, seasonality_modes, seasonality_prior_scales))

# Fungsi untuk menghitung SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Menyimpan hasil tuning
best_smape = float('inf')
best_params = None

# Train dan test split
train_size = int(0.8 * len(df_Revenue))
train_data = df_Revenue[:train_size]
test_data = df_Revenue[train_size:]

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_mode, seasonality_prior_scale in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        # weekly_seasonality=False, 
        # daily_seasonality=False,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_mode=seasonality_mode,
        seasonality_prior_scale=seasonality_prior_scale
    )
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    
    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='MS')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, seasonality_mode={seasonality_mode}, seasonality_prior_scale={seasonality_prior_scale}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_mode, seasonality_prior_scale)

print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, seasonality_mode={best_params[1]}, seasonality_prior_scale={best_params[2]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True, 
    # weekly_seasonality=False, 
    # daily_seasonality=False,
    changepoint_prior_scale=best_params[0],
    seasonality_mode=best_params[1],
    seasonality_prior_scale=best_params[2]
)
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(df_Revenue)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=12, freq='MS')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_df_Revenue = forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_df_Revenue.columns = ['Date', 'Forecast', 'Lower Bound', 'Upper Bound']

forecast_df_Revenue['Forecast'] = forecast_df_Revenue['Forecast'].round(2)
forecast_df_Revenue['Lower Bound'] = forecast_df_Revenue['Lower Bound'].round(2)
forecast_df_Revenue['Upper Bound'] = forecast_df_Revenue['Upper Bound'].round(2)

# Menampilkan DataFrame hasil prediksi
# forecast_df_Revenue.tail(16)


10:42:22 - cmdstanpy - INFO - Chain [1] start processing
10:42:22 - cmdstanpy - INFO - Chain [1] done processing
10:42:22 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.05, seasonality_mode=additive, seasonality_prior_scale=0.1: 15.35%


10:42:22 - cmdstanpy - INFO - Chain [1] done processing
10:42:22 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.05, seasonality_mode=additive, seasonality_prior_scale=0.2: 15.73%


10:42:23 - cmdstanpy - INFO - Chain [1] done processing
10:42:23 - cmdstanpy - INFO - Chain [1] start processing


...

SMAPE dengan changepoint_prior_scale=0.2, seasonality_mode=multiplicative, seasonality_prior_scale=0.1: 13.46%


10:42:26 - cmdstanpy - INFO - Chain [1] done processing
10:42:26 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.2, seasonality_mode=multiplicative, seasonality_prior_scale=0.2: 24.47%

Parameter terbaik: changepoint_prior_scale=0.2, seasonality_mode=multiplicative, seasonality_prior_scale=0.1
SMAPE terbaik: 13.46%


10:42:27 - cmdstanpy - INFO - Chain [1] done processing


           ds          yhat    yhat_lower    yhat_upper
0  2022-01-01  2.618109e+12  2.164468e+12  3.064125e+12
1  2022-02-01  2.086784e+12  1.642332e+12  2.523612e+12
2  2022-03-01  3.976129e+12  3.522393e+12  4.424355e+12
3  2022-04-01  3.633095e+12  3.197823e+12  4.046846e+12
4  2022-05-01  3.522459e+12  3.095702e+12  3.933360e+12
5  2022-06-01  3.364740e+12  2.955750e+12  3.799515e+12
6  2022-07-01  3.878877e+12  3.438003e+12  4.334392e+12
7  2022-08-01  3.891201e+12  3.469455e+12  4.327350e+12
8  2022-09-01  4.048963e+12  3.601529e+12  4.484942e+12
9  2022-10-01  4.457266e+12  4.030969e+12  4.885806e+12
10 2022-11-01  3.377257e+12  2.958495e+12  3.819278e+12
11 2022-12-01  3.873263e+12  3.429220e+12  4.301316e+12
12 2023-01-01  3.174789e+12  2.761659e+12  3.596750e+12
13 2023-02-01  2.643153e+12  2.216013e+12  3.102789e+12
14 2023-03-01  3.464732e+12  3.036496e+12  3.917354e+12
15 2023-04-01  3.289745e+12  2.867128e+12  3.734269e+12
16 2023-05-01  2.884714e+12  2.433249e+12  3.315

In [46]:
forecast_df_Revenue.loc[:, 'Date'] = pd.to_datetime(forecast_df_Revenue['Date'])
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'ds': forecast_df_Revenue['Date'],
    'y': forecast_df_Revenue['Forecast'],
    'jenis': 'forecast',
    'Breakdown' : 'Total Revenue'
})

# Menambahkan data untuk 'Lower Bound'
lower_bound_rows = pd.DataFrame({
    'ds': forecast_df_Revenue['Date'],
    'y': forecast_df_Revenue['Lower Bound'],
    'jenis': 'lower bound',
    'Breakdown' : 'Total Revenue'
})

# Menambahkan data untuk 'Upper Bound'
upper_bound_rows = pd.DataFrame({
    'ds': forecast_df_Revenue['Date'],
    'y': forecast_df_Revenue['Upper Bound'],
    'jenis': 'upper bound',
    'Breakdown' : 'Total Revenue'
})
df_Revenue_actual = pd.DataFrame({
    'ds': df_Revenue['ds'],
    'y': df_Revenue['y'],
    'jenis': 'actual',
    'Breakdown' : 'Total Revenue'
})
# Menggabungkan semua DataFrame
result_df_Revenue = pd.concat([df_Revenue_actual,forecast_rows, lower_bound_rows, upper_bound_rows], ignore_index=True)

# Menampilkan hasil akhir
# result_df_Revenue

----

In [47]:
print("--"*60)
print("CASH AND DEPOSIT")

------------------------------------------------------------------------------------------------------------------------
CASH AND DEPOSIT


In [48]:
df_Cash_Deposit = df_overview[df_overview['BreakDown2']=='Cash and Deposit'][['Date','amount']]
# Reset index
df_Cash_Deposit = df_Cash_Deposit.reset_index(drop=True)
df_Cash_Deposit.columns = ['ds','y']
df_Cash_Deposit['ds'] = pd.to_datetime(df_Cash_Deposit['ds'])


In [49]:
#  Kombinasi parameter untuk tuning SARIMA
p = d = q = range(0, 2)  # AR, I, MA order
P = D = Q = range(0, 2)  # Seasonal ARIMA order
s = [12]  # Musiman dalam bulan (12 bulan)

# Membuat list kombinasi parameter p, d, q, P, D, Q, s
param_combinations = list(itertools.product(p, d, q, P, D, Q, s))

# Menyimpan hasil tuning
best_smape = float('inf')
best_params = None

# Train dan test split
train_size = int(0.8 * len(df_Cash_Deposit))
train_data = df_Cash_Deposit[:train_size]
test_data = df_Cash_Deposit[train_size:]

# Fungsi SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Looping untuk setiap kombinasi parameter
for (p, d, q, P, D, Q, s) in param_combinations:
    try:
        # Inisialisasi model SARIMA dengan parameter tuning
        model = SARIMAX(train_data['y'],
                        order=(p, d, q),
                        seasonal_order=(P, D, Q, s),
                        enforce_stationarity=False,
                        enforce_invertibility=False)

        # Melatih model dengan data train
        model_fit = model.fit(disp=False)

        # Melakukan prediksi
        forecast = model_fit.forecast(steps=len(test_data))

        # Menggabungkan data asli dengan prediksi
        test_data.loc[:, 'yhat'] = forecast

        # Menghitung SMAPE untuk data test
        smape_value = smape(test_data['y'], test_data['yhat'])
        print(f"SMAPE dengan order=({p},{d},{q}), seasonal_order=({P},{D},{Q},{s}): {smape_value:.2f}%")

        # Menyimpan kombinasi parameter terbaik
        if smape_value < best_smape:
            best_smape = smape_value
            best_params = (p, d, q, P, D, Q, s)

    except Exception as e:
        print(f"Error untuk parameter {p, d, q, P, D, Q, s}: {e}")

print(f"\nParameter terbaik: order=({best_params[0]},{best_params[1]},{best_params[2]}), seasonal_order=({best_params[3]},{best_params[4]},{best_params[5]},{best_params[6]})")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Membuat model SARIMAX menggunakan parameter terbaik
final_model = SARIMAX(df_Cash_Deposit['y'],
                     order=(best_params[0], best_params[1], best_params[2]),
                     seasonal_order=(best_params[3], best_params[4], best_params[5], best_params[6]),
                     enforce_stationarity=False,
                     enforce_invertibility=False)

# Melatih model dengan seluruh data
final_model_fit = final_model.fit(disp=False)

# Prediksi untuk seluruh data historis (dari awal data hingga akhir)
forecast_historic = final_model_fit.get_prediction(steps=len(df_Cash_Deposit))
forecast_historic_mean = forecast_historic.predicted_mean
forecast_historic_ci = forecast_historic.conf_int()

# Prediksi untuk 12 bulan ke depan
forecast_future = final_model_fit.get_forecast(steps=12)
forecast_future_mean = forecast_future.predicted_mean
forecast_future_ci = forecast_future.conf_int()

# Membuat tanggal untuk prediksi masa depan
future_dates = pd.date_range(start=df_Cash_Deposit['ds'].max() + pd.DateOffset(months=1), periods=12, freq='MS')

# Konversi 'future_dates' ke Series agar bisa digabungkan dengan tanggal historis
future_dates_series = pd.Series(future_dates)

# Menyimpan hasil prediksi ke dalam DataFrame untuk data historis dan data masa depan
forecast_df_Cash_Deposit = pd.DataFrame({
    'Date': pd.concat([df_Cash_Deposit['ds'].reset_index(drop=True), future_dates_series], ignore_index=True),
    'Forecast': pd.concat([forecast_historic_mean, forecast_future_mean], ignore_index=True),
    'Lower Bound': pd.concat([forecast_historic_ci.iloc[:, 0], forecast_future_ci.iloc[:, 0]], ignore_index=True),
    'Upper Bound': pd.concat([forecast_historic_ci.iloc[:, 1], forecast_future_ci.iloc[:, 1]], ignore_index=True)
})

forecast_df_Cash_Deposit['Forecast'] = forecast_df_Cash_Deposit['Forecast'].round(2)
forecast_df_Cash_Deposit['Lower Bound'] = forecast_df_Cash_Deposit['Lower Bound'].round(2)
forecast_df_Cash_Deposit['Upper Bound'] = forecast_df_Cash_Deposit['Upper Bound'].round(2)

# Menampilkan hasil prediksi untuk seluruh data (historis + masa depan)
# forecast_df_Cash_Deposit

SMAPE dengan order=(0,0,0), seasonal_order=(0,0,0,12): 200.00%
SMAPE dengan order=(0,0,0), seasonal_order=(0,0,1,12): 76.60%
SMAPE dengan order=(0,0,0), seasonal_order=(0,1,0,12): 83.71%
SMAPE dengan order=(0,0,0), seasonal_order=(0,1,1,12): 83.71%
SMAPE dengan order=(0,0,0), seasonal_order=(1,0,0,12): 72.95%
SMAPE dengan order=(0,0,0), seasonal_order=(1,0,1,12): 81.00%
SMAPE dengan order=(0,0,0), seasonal_order=(1,1,0,12): 138.68%
SMAPE dengan order=(0,0,0), seasonal_order=(1,1,1,12): 83.71%
SMAPE dengan order=(0,0,1), seasonal_order=(0,0,0,12): 200.00%
SMAPE dengan order=(0,0,1), seasonal_order=(0,0,1,12): 73.21%
SMAPE dengan order=(0,0,1), seasonal_order=(0,1,0,12): 80.64%
SMAPE dengan order=(0,0,1), seasonal_order=(0,1,1,12): 80.37%
SMAPE dengan order=(0,0,1), seasonal_order=(1,0,0,12): 66.81%
SMAPE dengan order=(0,0,1), seasonal_order=(1,0,1,12): 66.76%
SMAPE dengan order=(0,0,1), seasonal_order=(1,1,0,12): 138.68%
SMAPE dengan order=(0,0,1), seasonal_order=(1,1,1,12): 80.37%
SMAP

In [50]:
forecast_df_Cash_Deposit.loc[:, 'Date'] = pd.to_datetime(forecast_df_Cash_Deposit['Date'])
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'ds': forecast_df_Cash_Deposit['Date'],
    'y': forecast_df_Cash_Deposit['Forecast'],
    'jenis': 'forecast',
    'Breakdown' : 'Cash and Deposit'
})

# Menambahkan data untuk 'Lower Bound'
lower_bound_rows = pd.DataFrame({
    'ds': forecast_df_Cash_Deposit['Date'],
    'y': forecast_df_Cash_Deposit['Lower Bound'],
    'jenis': 'lower bound',
    'Breakdown' : 'Cash and Deposit'
})

# Menambahkan data untuk 'Upper Bound'
upper_bound_rows = pd.DataFrame({
    'ds': forecast_df_Cash_Deposit['Date'],
    'y': forecast_df_Cash_Deposit['Upper Bound'],
    'jenis': 'upper bound',
    'Breakdown' : 'Cash and Deposit'
})
df_Cash_Deposit_actual = pd.DataFrame({
    'ds': df_Cash_Deposit['ds'],
    'y': df_Cash_Deposit['y'],
    'jenis': 'actual',
    'Breakdown' : 'Cash and Deposit'
})
# Menggabungkan semua DataFrame
result_df_Cash_Deposit = pd.concat([df_Cash_Deposit_actual,forecast_rows, lower_bound_rows, upper_bound_rows], ignore_index=True)

# Menampilkan hasil akhir
# result_df_Cash_Deposit

---

In [51]:
print("--"*60)
print("EBITDA")

------------------------------------------------------------------------------------------------------------------------
EBITDA


In [52]:
df_EBITDA = df_overview[df_overview['BreakDown2']=='EBITDA After Minority'][['Date','amount']]
# Reset index
df_EBITDA = df_EBITDA.reset_index(drop=True)
df_EBITDA.columns = ['ds','y']
df_EBITDA['ds'] = pd.to_datetime(df_EBITDA['ds'])


In [53]:
# Kombinasi parameter untuk tuning
changepoint_prior_scales = [ 0.1, 0.2]
seasonality_modes = ['additive','multiplicative']
seasonality_prior_scales = [0.1, 0.2]

# Membuat list kombinasi parameter
param_combinations = list(itertools.product(changepoint_prior_scales, seasonality_modes, seasonality_prior_scales))

# Fungsi untuk menghitung SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Menyimpan hasil tuning
best_smape = float('inf')
best_params = None

# Train dan test split
train_size = int(0.8 * len(df_EBITDA))
train_data = df_EBITDA[:train_size]
test_data = df_EBITDA[train_size:]

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_mode, seasonality_prior_scale in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        # weekly_seasonality=False, 
        # daily_seasonality=False,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_mode=seasonality_mode,
        seasonality_prior_scale=seasonality_prior_scale
    )
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    
    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='MS')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, seasonality_mode={seasonality_mode}, seasonality_prior_scale={seasonality_prior_scale}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_mode, seasonality_prior_scale)

print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, seasonality_mode={best_params[1]}, seasonality_prior_scale={best_params[2]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True, 
    # weekly_seasonality=False, 
    # daily_seasonality=False,
    changepoint_prior_scale=best_params[0],
    seasonality_mode=best_params[1],
    seasonality_prior_scale=best_params[2]
)
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(df_EBITDA)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=12, freq='MS')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_df_EBITDA = forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_df_EBITDA.columns = ['Date', 'Forecast', 'Lower Bound', 'Upper Bound']

forecast_df_EBITDA['Forecast'] = forecast_df_EBITDA['Forecast'].round(2)
forecast_df_EBITDA['Lower Bound'] = forecast_df_EBITDA['Lower Bound'].round(2)
forecast_df_EBITDA['Upper Bound'] = forecast_df_EBITDA['Upper Bound'].round(2)

# Menampilkan DataFrame hasil prediksi
# forecast_df_EBITDA.tail(16)


10:42:28 - cmdstanpy - INFO - Chain [1] start processing
10:42:28 - cmdstanpy - INFO - Chain [1] done processing
10:42:29 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.1, seasonality_mode=additive, seasonality_prior_scale=0.1: 73.02%


10:42:29 - cmdstanpy - INFO - Chain [1] done processing
10:42:29 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.1, seasonality_mode=additive, seasonality_prior_scale=0.2: 118.00%


10:42:29 - cmdstanpy - INFO - Chain [1] done processing
10:42:29 - cmdstanpy - INFO - Chain [1] start processing


...

SMAPE dengan changepoint_prior_scale=0.2, seasonality_mode=multiplicative, seasonality_prior_scale=0.1: 32.97%


10:42:31 - cmdstanpy - INFO - Chain [1] done processing
10:42:31 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.2, seasonality_mode=multiplicative, seasonality_prior_scale=0.2: 25.46%

Parameter terbaik: changepoint_prior_scale=0.1, seasonality_mode=multiplicative, seasonality_prior_scale=0.2
SMAPE terbaik: 25.26%


10:42:32 - cmdstanpy - INFO - Chain [1] done processing


           ds          yhat    yhat_lower    yhat_upper
0  2022-01-01  8.490543e+11  4.354804e+11  1.262015e+12
1  2022-02-01  7.180133e+11  2.993661e+11  1.112087e+12
2  2022-03-01  1.638697e+12  1.255959e+12  2.067227e+12
3  2022-04-01  1.436283e+12  1.018249e+12  1.840470e+12
4  2022-05-01  1.471993e+12  1.066878e+12  1.871927e+12
5  2022-06-01  1.996983e+12  1.582251e+12  2.378661e+12
6  2022-07-01  1.635411e+12  1.219615e+12  2.034265e+12
7  2022-08-01  1.666735e+12  1.288968e+12  2.082262e+12
8  2022-09-01  1.407867e+12  9.824776e+11  1.800581e+12
9  2022-10-01  1.784378e+12  1.383766e+12  2.202061e+12
10 2022-11-01  1.071412e+12  6.370304e+11  1.468123e+12
11 2022-12-01  1.686471e+12  1.282515e+12  2.079301e+12
12 2023-01-01  7.294915e+11  3.007409e+11  1.143651e+12
13 2023-02-01  6.858733e+11  2.854771e+11  1.098108e+12
14 2023-03-01  9.494826e+11  5.314161e+11  1.347264e+12
15 2023-04-01  8.509171e+11  4.468798e+11  1.268438e+12
16 2023-05-01  8.243567e+11  4.367776e+11  1.229

In [54]:
forecast_df_EBITDA.loc[:, 'Date'] = pd.to_datetime(forecast_df_EBITDA['Date'])
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'ds': forecast_df_EBITDA['Date'],
    'y': forecast_df_EBITDA['Forecast'],
    'jenis': 'forecast',
    'Breakdown' : 'EBITDA After Minority'
})

# Menambahkan data untuk 'Lower Bound'
lower_bound_rows = pd.DataFrame({
    'ds': forecast_df_EBITDA['Date'],
    'y': forecast_df_EBITDA['Lower Bound'],
    'jenis': 'lower bound',
    'Breakdown' : 'EBITDA After Minority'
})

# Menambahkan data untuk 'Upper Bound'
upper_bound_rows = pd.DataFrame({
    'ds': forecast_df_EBITDA['Date'],
    'y': forecast_df_EBITDA['Upper Bound'],
    'jenis': 'upper bound',
    'Breakdown' : 'EBITDA After Minority'
})
df_EBITDA_actual = pd.DataFrame({
    'ds': df_EBITDA['ds'],
    'y': df_EBITDA['y'],
    'jenis': 'actual',
    'Breakdown' : 'EBITDA After Minority'
})
# Menggabungkan semua DataFrame
result_df_EBITDA = pd.concat([df_EBITDA_actual,forecast_rows, lower_bound_rows, upper_bound_rows], ignore_index=True)

# Menampilkan hasil akhir
# result_df_EBITDA

---

In [55]:
print("--"*60)
print("NET PROFIT")

------------------------------------------------------------------------------------------------------------------------
NET PROFIT


In [56]:
df_Netprofit = df_overview[df_overview['BreakDown2']=='Net Profit After Minority'][['Date','amount']]
# Reset index
df_Netprofit = df_Netprofit.reset_index(drop=True)
df_Netprofit.columns = ['ds','y']
df_Netprofit['ds'] = pd.to_datetime(df_Netprofit['ds'])


In [57]:
# Kombinasi parameter untuk tuning
changepoint_prior_scales = [ 0.1,0.2, 0.5]
seasonality_modes = ['multiplicative']
seasonality_prior_scales = [ 0.1,0.2]

# Membuat list kombinasi parameter
param_combinations = list(itertools.product(changepoint_prior_scales, seasonality_modes, seasonality_prior_scales))

# Fungsi untuk menghitung SMAPE
def smape(actual, predicted):
    return 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(predicted) + np.abs(actual)))

# Menyimpan hasil tuning
best_smape = float('inf')
best_params = None

# Train dan test split
train_size = int(0.8 * len(df_Netprofit))
train_data = df_Netprofit[:train_size]
test_data = df_Netprofit[train_size:]

# Looping untuk setiap kombinasi parameter
for changepoint_prior_scale, seasonality_mode, seasonality_prior_scale in param_combinations:
    
    # Inisialisasi model Prophet dengan parameter tuning
    model = Prophet(
        yearly_seasonality=True, 
        # weekly_seasonality=False, 
        # daily_seasonality=False,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_mode=seasonality_mode,
        seasonality_prior_scale=seasonality_prior_scale
    )
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    
    # Melatih model dengan data train
    model.fit(train_data)

    # Membuat dataframe masa depan yang mencakup periode data test
    future = model.make_future_dataframe(periods=len(test_data), freq='MS')

    # Melakukan prediksi
    forecast = model.predict(future)

    # Mengambil prediksi yang sesuai dengan data test
    forecast_test = forecast[-len(test_data):]

    # Menggabungkan data asli dengan prediksi
    test_data.loc[:, 'yhat'] = forecast_test['yhat'].values

    # Menghitung SMAPE untuk data test
    smape_value = smape(test_data['y'], test_data['yhat'])
    print(f"SMAPE dengan changepoint_prior_scale={changepoint_prior_scale}, seasonality_mode={seasonality_mode}, seasonality_prior_scale={seasonality_prior_scale}: {smape_value:.2f}%")

    # Menyimpan kombinasi parameter terbaik
    if smape_value < best_smape:
        best_smape = smape_value
        best_params = (changepoint_prior_scale, seasonality_mode, seasonality_prior_scale)

print(f"\nParameter terbaik: changepoint_prior_scale={best_params[0]}, seasonality_mode={best_params[1]}, seasonality_prior_scale={best_params[2]}")
print(f"SMAPE terbaik: {best_smape:.2f}%")

# Latih model dengan seluruh data menggunakan parameter terbaik
final_model = Prophet(
    yearly_seasonality=True, 
    # weekly_seasonality=False, 
    # daily_seasonality=False,
    changepoint_prior_scale=best_params[0],
    seasonality_mode=best_params[1],
    seasonality_prior_scale=best_params[2]
)
final_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# Melatih model dengan seluruh data
final_model.fit(df_Netprofit)

# Membuat dataframe masa depan untuk 6 bulan ke depan
future = final_model.make_future_dataframe(periods=12, freq='MS')

# Melakukan prediksi
forecast_final = final_model.predict(future)

# Menampilkan hasil prediksi
print(forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Menyimpan hasil prediksi ke dalam DataFrame
forecast_df_Netprofit = forecast_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_df_Netprofit.columns = ['Date', 'Forecast', 'Lower Bound', 'Upper Bound']

forecast_df_Netprofit['Forecast'] = forecast_df_Netprofit['Forecast'].round(2)
forecast_df_Netprofit['Lower Bound'] = forecast_df_Netprofit['Lower Bound'].round(2)
forecast_df_Netprofit['Upper Bound'] = forecast_df_Netprofit['Upper Bound'].round(2)

# Menampilkan DataFrame hasil prediksi
# forecast_df_Netprofit.tail(16)


10:42:32 - cmdstanpy - INFO - Chain [1] start processing
10:42:32 - cmdstanpy - INFO - Chain [1] done processing
10:42:33 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.1, seasonality_mode=multiplicative, seasonality_prior_scale=0.1: 65.12%


10:42:33 - cmdstanpy - INFO - Chain [1] done processing
10:42:33 - cmdstanpy - INFO - Chain [1] start processing


SMAPE dengan changepoint_prior_scale=0.1, seasonality_mode=multiplicative, seasonality_prior_scale=0.2: 34.86%


10:42:33 - cmdstanpy - INFO - Chain [1] done processing
10:42:33 - cmdstanpy - INFO - Chain [1] start processing


...

SMAPE dengan changepoint_prior_scale=0.5, seasonality_mode=multiplicative, seasonality_prior_scale=0.1: 105.10%


10:42:35 - cmdstanpy - INFO - Chain [1] done processing


SMAPE dengan changepoint_prior_scale=0.5, seasonality_mode=multiplicative, seasonality_prior_scale=0.2: 107.59%

Parameter terbaik: changepoint_prior_scale=0.2, seasonality_mode=multiplicative, seasonality_prior_scale=0.1
SMAPE terbaik: 29.57%


10:42:35 - cmdstanpy - INFO - Chain [1] start processing
10:42:36 - cmdstanpy - INFO - Chain [1] done processing


           ds          yhat    yhat_lower    yhat_upper
0  2022-01-01  6.452457e+11  3.789889e+11  9.209044e+11
1  2022-02-01  5.895082e+11  2.996405e+11  8.602434e+11
2  2022-03-01  1.232327e+12  9.640075e+11  1.501016e+12
3  2022-04-01  1.094250e+12  8.204930e+11  1.376582e+12
4  2022-05-01  1.063293e+12  7.782563e+11  1.356299e+12
5  2022-06-01  1.550572e+12  1.266719e+12  1.829041e+12
6  2022-07-01  1.151900e+12  8.815633e+11  1.455781e+12
7  2022-08-01  1.190253e+12  9.204259e+11  1.473096e+12
8  2022-09-01  9.303500e+11  6.384204e+11  1.192580e+12
9  2022-10-01  1.143684e+12  8.630533e+11  1.422898e+12
10 2022-11-01  6.711845e+11  3.953790e+11  9.535840e+11
11 2022-12-01  1.064996e+12  7.952850e+11  1.350133e+12
12 2023-01-01  4.691389e+11  1.789876e+11  7.494920e+11
13 2023-02-01  4.164738e+11  1.328671e+11  7.136161e+11
14 2023-03-01  6.457488e+11  3.523296e+11  9.161235e+11
15 2023-04-01  5.854394e+11  2.932220e+11  8.532915e+11
16 2023-05-01  5.184572e+11  2.209382e+11  7.947

In [58]:
forecast_df_Netprofit.loc[:, 'Date'] = pd.to_datetime(forecast_df_Netprofit['Date'])
# Menambahkan data untuk 'Forecast'
forecast_rows = pd.DataFrame({
    'ds': forecast_df_Netprofit['Date'],
    'y': forecast_df_Netprofit['Forecast'],
    'jenis': 'forecast',
    'Breakdown' : 'Net Profit After Minority'
})

# Menambahkan data untuk 'Lower Bound'
lower_bound_rows = pd.DataFrame({
    'ds': forecast_df_Netprofit['Date'],
    'y': forecast_df_Netprofit['Lower Bound'],
    'jenis': 'lower bound',
    'Breakdown' : 'Net Profit After Minority'
})

# Menambahkan data untuk 'Upper Bound'
upper_bound_rows = pd.DataFrame({
    'ds': forecast_df_Netprofit['Date'],
    'y': forecast_df_Netprofit['Upper Bound'],
    'jenis': 'upper bound',
    'Breakdown' : 'Net Profit After Minority'
})
df_Netprofit_actual = pd.DataFrame({
    'ds': df_Netprofit['ds'],
    'y': df_Netprofit['y'],
    'jenis': 'actual',
    'Breakdown' : 'Net Profit After Minority'
})
# Menggabungkan semua DataFrame
result_df_Netprofit = pd.concat([df_Netprofit_actual,forecast_rows, lower_bound_rows, upper_bound_rows], ignore_index=True)

# Menampilkan hasil akhir
# result_df_Netprofit

---

In [59]:
print("--"*60)
print("DF FORECAST")

------------------------------------------------------------------------------------------------------------------------
DF FORECAST


In [60]:
df_Forecast_all = pd.concat([result_df_production,result_df_Transport,result_df_SR, result_df_Sales, result_df_coal_cast_cost, result_df_Avg_SP, result_df_Revenue, result_df_Cash_Deposit,result_df_EBITDA, result_df_Netprofit],ignore_index=True)
# df_Forecast_all = pd.concat([result_df_production,result_df_Transport,result_df_SR, result_df_Sales, result_df_coal_cast_cost, result_df_Avg_SP],ignore_index=True)
df_Forecast_all.rename(columns={'ds': 'Date', 'y': 'Amount', 'jenis':'Tipe','Breakdown':'BreakDown'}, inplace=True)
# df_Forecast_all

In [61]:
from datetime import datetime
df_Forecast_all['last_updated'] = datetime.now()
df_Forecast_all['Updated_Date'] = datetime.now()
# df_Forecast_all

---

In [62]:
print("--"*60)
print("DF LAMA")

------------------------------------------------------------------------------------------------------------------------
DF LAMA


In [63]:
df_overview_filter.copy()

,Date,bulan,tahun,Indicator,BreakDown,Unit,Country,Source,tipe,amount,last_updated,Updated_Date,BreakDown2
11,2022-01-01,1,2022,None,Cash and Deposit,None,None,None,Worst,0.00,NaT,NaT,Cash and Deposit
25,2022-01-01,1,2022,None,Coal Cash Cost,None,None,None,Moderate,0.00,NaT,NaT,Coal Cash Cost
26,2022-01-01,1,2022,None,Coal Cash Cost,None,None,None,Worst,0.00,NaT,NaT,Coal Cash Cost
103,2022-01-01,1,2022,None,Average Market Price IDR,None,None,None,Actual,0.00,NaT,NaT,Average Market Price IDR
104,2022-01-01,1,2022,None,Average Market Price IDR,None,None,None,Worst,0.00,NaT,NaT,Average Market Price IDR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69019,2024-08-01,8,2024,External Data,Average Market Price IDR,IDR/Ton,None,None,RKAP,983539.31,2024-08-31 05:11:01,2024-10-08 05:00:07.107,Average Market Price IDR
69020,2024-08-01,8,2024,External Data,Average Market Price IDR,IDR/Ton,None,None,Optimize,0.00,2024-08-31 05:11:01,2024-10-08 05:00:07.107,Average Market Price IDR
69021,2024-08-01,8,2024,External Data,Average Market Price IDR,IDR/Ton,None,None,Moderate,0.00,2024-08-31 05:11:01,2024-10-08 05:00:07.107,Average Market Price IDR
69022,2024-08-01,8,2024,External Data,Average Market Price IDR,IDR/Ton,None,None,Worst,0.00,2024-08-31 05:11:01,2024-10-08 05:00:07.107,Average Market Price IDR


In [64]:
df_overview_non_actual = df_overview_filter[df_overview_filter['tipe'] != "Actual"]
df_overview_non_actual['Date'] = pd.to_datetime(df_overview_non_actual['Date'])
# df_overview_non_actual

In [65]:
df_overview_non_Actual_group = df_overview_non_actual.groupby(['Date','BreakDown2','tipe','last_updated','Updated_Date'])['amount'].sum().reset_index()
df_overview_non_Actual_group.rename(columns={'amount': 'Amount', 'tipe':'Tipe','BreakDown2':'BreakDown'}, inplace=True)
# df_overview_non_Actual_group

In [66]:
print("--"*60)
print("INSERT DATA TO DATABASE")

------------------------------------------------------------------------------------------------------------------------
INSERT DATA TO DATABASE


In [67]:
# gabungkan semua
df_Final = pd.concat([df_Forecast_all, df_overview_non_Actual_group],ignore_index=True)
df_Final.rename(columns={'Amount': 'amount', 'Tipe':'tipe'}, inplace=True)
df_Final['amount'] = df_Final['amount'].round(2)  # Membatasi hingga 2 angka desimal
ordered_columns = ['Date', 'BreakDown', 'tipe', 'amount', 'last_updated', 'Updated_Date']
df_Final['amount'] = df_Final['amount'].astype(float)
df_Final = df_Final.reindex(columns=ordered_columns)
df_Final

,Date,BreakDown,tipe,amount,last_updated,Updated_Date
0,2022-01-01,Total Production,actual,1888965.44,2024-10-08 10:42:36.369646,2024-10-08 10:42:36.370508
1,2022-02-01,Total Production,actual,1894946.46,2024-10-08 10:42:36.369646,2024-10-08 10:42:36.370508
2,2022-03-01,Total Production,actual,2554719.12,2024-10-08 10:42:36.369646,2024-10-08 10:42:36.370508
3,2022-04-01,Total Production,actual,2661113.36,2024-10-08 10:42:36.369646,2024-10-08 10:42:36.370508
4,2022-05-01,Total Production,actual,3112058.29,2024-10-08 10:42:36.369646,2024-10-08 10:42:36.370508
...,...,...,...,...,...,...
3235,2024-08-01,Total Transport,Moderate,0.00,2024-08-31 05:11:01.000000,2024-10-08 05:00:07.107000
3236,2024-08-01,Total Transport,Optimize,0.00,2024-08-31 05:11:01.000000,2024-10-08 05:00:07.107000
3237,2024-08-01,Total Transport,Projection,0.00,2024-08-31 05:11:01.000000,2024-10-08 05:00:07.107000
3238,2024-08-01,Total Transport,RKAP,2875500.00,2024-08-31 05:11:01.000000,2024-10-08 05:00:07.107000


In [68]:
df_Final[['last_updated', 'Updated_Date']] = df_Final[['last_updated', 'Updated_Date']].apply(pd.to_datetime)


In [70]:
df_Final

,Date,BreakDown,tipe,amount,last_updated,Updated_Date
0,2022-01-01,Total Production,actual,1888965.44,2024-10-08 10:42:36.369646,2024-10-08 10:42:36.370508
1,2022-02-01,Total Production,actual,1894946.46,2024-10-08 10:42:36.369646,2024-10-08 10:42:36.370508
2,2022-03-01,Total Production,actual,2554719.12,2024-10-08 10:42:36.369646,2024-10-08 10:42:36.370508
3,2022-04-01,Total Production,actual,2661113.36,2024-10-08 10:42:36.369646,2024-10-08 10:42:36.370508
4,2022-05-01,Total Production,actual,3112058.29,2024-10-08 10:42:36.369646,2024-10-08 10:42:36.370508
...,...,...,...,...,...,...
3235,2024-08-01,Total Transport,Moderate,0.00,2024-08-31 05:11:01.000000,2024-10-08 05:00:07.107000
3236,2024-08-01,Total Transport,Optimize,0.00,2024-08-31 05:11:01.000000,2024-10-08 05:00:07.107000
3237,2024-08-01,Total Transport,Projection,0.00,2024-08-31 05:11:01.000000,2024-10-08 05:00:07.107000
3238,2024-08-01,Total Transport,RKAP,2875500.00,2024-08-31 05:11:01.000000,2024-10-08 05:00:07.107000


In [83]:
import pyodbc
from dotenv import load_dotenv
import os
import pandas as pd

# Muat variabel lingkungan dari file .env
# load_dotenv()
# server = os.environ['SERVER']
# database = os.environ['DATABASE']
# username = os.environ['USERNAME_1']
# password = os.environ['PASSWORD']

# SERVER=10.3.4.139,1433
# DATABASE=dwh_prod
# USERNAME_1=developerptba
# PASSWORD=LnfPYVFW1K2TAKf
# Hardcode nilai koneksi SQL Server
server = '10.3.4.139,1433'
database = 'dwh_prod'
username = 'developerptba'
password = 'LnfPYVFW1K2TAKf'

# Koneksi ke SQL Server
try:
    # conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
    print("Connection successful")
    
    cursor = conn.cursor()

    # Truncate tabel
    truncate_query = "TRUNCATE TABLE dwh.DM_forecast_financial_position"
    cursor.execute(truncate_query)
    print("Table truncated successfully.")

    # Setting fast_executemany
    cursor.fast_executemany = True

    insert_query = """
    INSERT INTO dwh.DM_forecast_financial_position 
    ([Date], [BreakDown], [tipe], [amount], [last_updated], [Updated_Date]) 
    VALUES (?, ?, ?, ?, ?, ?)
    """

    # Siapkan data yang akan di-insert
    rows_to_insert = [row for row in df_Final.itertuples(index=False)]
    print(f"{len(rows_to_insert)} rows will be inserted into the database.")

    # Nonaktifkan autocommit
    conn.autocommit = False

    # Menggunakan try-except saat insert
    for row in rows_to_insert:
        cursor.execute(insert_query, row)
        
    # Commit perubahan jika semua insert berhasil
    conn.commit()
    print("Data successfully inserted.")

except Exception as e:
    print(f"An error occurred: {e}")
    if conn:
        conn.rollback()  # Rollback jika terjadi error

finally:
    cursor.close()
    conn.close()  # Pastikan koneksi selalu ditutup
    print("Connection closed.")


Connection successful
Table truncated successfully.
3240 rows will be inserted into the database.
Data successfully inserted.
Connection closed.
